In [1]:
!pip install phonetics
!pip install epitran

  Preparing metadata (setup.py) ... done
  Created wheel for phonetics: filename=phonetics-1.0.5-py2.py3-none-any.whl size=8696 sha256=3159d0a7a398f00dbd24291af51167f84367ba6a7d91ca76e32f314be6fabf2c
  Stored in directory: /root/.cache/pip/wheels/b7/1e/82/80a78c7d1ad7fc6e0af1b4d9009360b251c0e50fe59f046edb
Successfully built phonetics
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.1 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10744 sha256=bd56e8bfd868e5580e72907ab543ab8a66fa2cb4f1553b0a14543ebcf627cf65
  Stored in directory: /root/.cache/pip/wheels/9c/ea/66/8e45247b09052a933eb1a680b7c64802298faba58aac9b346b
Successfully built unicodecsv


In [2]:
import pandas as pd
import numpy as np
import phonetics
import torch
import epitran
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
import pandas as pd
import os

# List of language pairs
language_pairs = [
    ("Azerbaijani", "Arabic"),
    ("Catalan", "Arabic"),
    ("Chinese", "English"),
    ("English", "French"),
    ("English", "German"),
    ("Finnish", "Swedish"),
    ("German", "French"),
    ("German", "Italian"),
    ("Hindi", "Persian"),
    ("Hungarian", "German"),
    ("Indonesian", "Dutch"),
    ("Kazakh", "Russian"),
    ("Persian", "Arabic"),
    ("Polish", "French"),
    ("Romanian", "French"),
    ("Romanian", "Hungarian"),
]

def read_language(lang1, lang2):
    file_path = f"/kaggle/input/dataset/Datasets/production_train_test/{lang1}-{lang2}/balanced/{lang1}-{lang2}-train_production_balanced.csv"    
    if os.path.exists(file_path):  
        df = pd.read_csv(file_path)
        df = df.drop(columns=[col for col in ['Unnamed: 0.1', 'Unnamed: 0'] if col in df.columns], errors="ignore")
        df["language_pair"] = f"{lang1}-{lang2}"
        return df
    else:
        print(f"File not found: {file_path}")
        return None

dfs = [read_language(lang1, lang2) for lang1, lang2 in language_pairs]
dfs = [df for df in dfs if df is not None]

final_df = pd.concat(dfs, ignore_index=True)

print(final_df.info())

final_df.to_csv("all_languages_combined.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47384 entries, 0 to 47383
Data columns (total 21 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   loan_word                                    47384 non-null  object 
 1   original_word                                47384 non-null  object 
 2   loan_word_epitran                            47384 non-null  object 
 3   original_word_epitran                        47384 non-null  object 
 4   loan_english                                 47159 non-null  object 
 5   original_english                             47175 non-null  object 
 6   Fast Levenshtein Distance Div Maxlen         47384 non-null  float64
 7   Dolgo Prime Distance Div Maxlen              47384 non-null  float64
 8   Feature Edit Distance Div Maxlen             47384 non-null  float64
 9   Hamming Feature Distance Div Maxlen          47384 non-null  float64
 10

In [4]:
# need to install - sudo apt-get install flite

epi = epitran.Epitran('fra-Latn')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def normalize(unicode_values):
    mean_value = sum(unicode_values) / len(unicode_values)
    return [val - mean_value for val in unicode_values]

def extract_features(word):
    try:
        loan_epitran = epi.transliterate(word)
        print(f"Transliterated text: {loan_epitran}")
    except IndexError as e:
        print(f"Transliteration failed: {e}")
        loan_epitran = "N/A"
    unicode_features = [ord(c) for c in word]
    unicode_features = normalize(unicode_features)
    return {
        "word": word,
        "phonetic": loan_epitran,
        "unicode": unicode_features,
        "length": len(word)
    }

extract_features("hello")

Transliterated text: ɛlo


{'word': 'hello',
 'phonetic': 'ɛlo',
 'unicode': [-2.4000000000000057,
  -5.400000000000006,
  1.5999999999999943,
  1.5999999999999943,
  4.599999999999994],
 'length': 5}

In [5]:
epi = epitran.Epitran('fra-Latn')

def normalize(unicode_values):
    """Normalize Unicode values by subtracting the mean."""
    mean_value = sum(unicode_values) / len(unicode_values)
    return [val - mean_value for val in unicode_values]

def extract_features(word):
    """Extract phonetic and Unicode-based features for a word."""
    try:
        loan_epitran = epi.transliterate(word)  # Phonetic transliteration
    except IndexError as e:
        print(f"Transliteration failed for '{word}': {e}")

    unicode_features = [ord(c) for c in word]  # Convert to Unicode
    unicode_features = normalize(unicode_features)  # Normalize Unicode values

    return {
        "phonetic": loan_epitran,
        "unicode": unicode_features,
        "length": len(word)
    }


df_features = final_df["loan_word"].apply(extract_features).apply(pd.Series)
df_features
df = pd.concat([final_df, df_features], axis=1)

In [6]:
df['label'] = df["label"].replace(['random', 'hard_negative', 'loan', 'synonym'],[0,1,0,0])
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")


def tokenize_and_encode(text, tokenizer, max_length=120):
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return encoding["input_ids"].squeeze(0), encoding["attention_mask"].squeeze(0)

df['phonetic_'], df['attention_masks'] = zip(*df['phonetic'].apply(
    lambda x: tokenize_and_encode(x, tokenizer)
))

def pad_sequence(seq, maxlen, pad_value=0):
    """Pads sequences with pad_value up to maxlen."""
    return seq + [pad_value] * (maxlen - len(seq))

max_unicode_len = 30

df['unicode_padded'] = df['unicode'].apply(lambda x: pad_sequence(x, max_unicode_len))


<ipython-input-6-54159fa235c6>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df["label"].replace(['random', 'hard_negative', 'loan', 'synonym'],[0,1,0,0])


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

text_pairs = list(zip(df["original_word"], df["loan_word"]))
labels = torch.tensor(df["label"].tolist())

inputs = tokenizer.batch_encode_plus(
    [f"{loan} [SEP] {original}" for loan, original in text_pairs],
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=128,
)

batch_size = 128
dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], labels)
dataloader2 = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

for name, param in model.bert.named_parameters():
    if not any(layer in name for layer in ["encoder.layer.9","encoder.layer.10", "encoder.layer.11"]):
        param.requires_grad = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)

for epoch in range(50):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader2, desc=f"Epoch {epoch+1}")

    for batch in progress_bar:
        input_ids, attention_mask, batch_labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=total_loss / (progress_bar.n + 1))

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader2)}")

model.save_pretrained("tuned-bert")
tokenizer.save_pretrained("tuned-bert-tokenizer")


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 371/371 [01:02<00:00,  5.97it/s, loss=0.228]


Epoch 1, Loss: 0.22834462734727526


Epoch 2: 100%|██████████| 371/371 [01:01<00:00,  5.99it/s, loss=0.193]


Epoch 2, Loss: 0.1933460118615563


...

Epoch 49: 100%|██████████| 371/371 [01:01<00:00,  5.99it/s, loss=0.0196]


Epoch 49, Loss: 0.019584939286802806


Epoch 50: 100%|██████████| 371/371 [01:01<00:00,  6.04it/s, loss=0.0195]


Epoch 50, Loss: 0.01951124999879902


('tuned-bert-tokenizer/tokenizer_config.json',
 'tuned-bert-tokenizer/special_tokens_map.json',
 'tuned-bert-tokenizer/vocab.txt',
 'tuned-bert-tokenizer/added_tokens.json')

In [9]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

model_path = "tuned-bert"
tokenizer_path = "tuned-bert-tokenizer"

tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model = BertForSequenceClassification.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to evaluation mode

def predict(loan_word, original_word):
    """Predicts if a loanword belongs to the target language."""
    
    encoded_input = tokenizer(
        f"{loan_word} [SEP] {original_word}",
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt",
    )

    input_ids = encoded_input["input_ids"].to(device)
    attention_mask = encoded_input["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()  # Convert to class label
    print(logits)
    return predicted_label

loan_word = "qəhbə"
original_word = "قَحْبَة"

predicted_class = predict(loan_word, original_word)
print(f"Prediction: {predicted_class}")


tensor([[ 6.3890, -6.8924]], device='cuda:0')
Prediction: 0


In [10]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Shape: {param.shape}")


Layer: bert.embeddings.word_embeddings.weight | Shape: torch.Size([119547, 768])
Layer: bert.embeddings.position_embeddings.weight | Shape: torch.Size([512, 768])
Layer: bert.embeddings.token_type_embeddings.weight | Shape: torch.Size([2, 768])
Layer: bert.embeddings.LayerNorm.weight | Shape: torch.Size([768])
Layer: bert.embeddings.LayerNorm.bias | Shape: torch.Size([768])
Layer: bert.encoder.layer.0.attention.self.query.weight | Shape: torch.Size([768, 768])
Layer: bert.encoder.layer.0.attention.self.query.bias | Shape: torch.Size([768])
Layer: bert.encoder.layer.0.attention.self.key.weight | Shape: torch.Size([768, 768])
Layer: bert.encoder.layer.0.attention.self.key.bias | Shape: torch.Size([768])
Layer: bert.encoder.layer.0.attention.self.value.weight | Shape: torch.Size([768, 768])
Layer: bert.encoder.layer.0.attention.self.value.bias | Shape: torch.Size([768])
Layer: bert.encoder.layer.0.attention.output.dense.weight | Shape: torch.Size([768, 768])
Layer: bert.encoder.layer.0.at

In [11]:
final_df

,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,...,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,loan_unicode,original_unicode,label,label_bin,DNN_logits,MBERT_cos_sim,XLM_cos_sim,language_pair
0,Möhkəmlik,البطولة,mœhkæmlɪk,albtˤuːlt,Durability,Championship,1.000000,0.666667,0.354167,0.393519,...,0.377315,9,Latin,Arabic,synonym,0,-18.639088,0.805898,0.519797,Azerbaijani-Arabic
1,mehriban,مرن,mehrɪbɑn,mrn,kind,flexible,0.625000,0.625000,0.557292,0.625000,...,0.625000,8,Latin,Arabic,synonym,0,-9.437688,0.833718,0.753124,Azerbaijani-Arabic
2,qəhbə,قَحْبَة,ɡæhbæ,qَħْbَt,bastard,whore,0.857143,0.600000,0.291667,0.325000,...,0.312500,7,Latin,Arabic,loan,1,4.729791,0.672404,0.517030,Azerbaijani-Arabic
3,təslim olmaq,التنازل عن العرش,tæslɪm ɔlmɑɡ,altnaːzl ʕn aːlʕrʃ,surrender,abdication,0.777778,0.571429,0.299107,0.333333,...,0.318452,15,Latin,Arabic,synonym,0,-49.171078,0.902082,0.500199,Azerbaijani-Arabic
4,tələsmək,مضطرب,tælæsmæk,mdˤtˤrb,hurry up,مضطرب,1.000000,0.875000,0.466146,0.520833,...,0.505208,8,Latin,Arabic,synonym,0,-7.328197,0.871732,0.551427,Azerbaijani-Arabic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47379,Barabas,Fellak,barabas,fɛlːɒk,Barabas,Fellak,1.000000,0.428571,0.351190,0.392857,...,0.377976,6,Latin,Latin,random,0,-10.796869,0.893563,0.532925,Romanian-Hungarian
47380,Agrișteu,Egrestő,aɡriʃteu,ɛɡrɛʃtøː,Agristeu,Egrestő,0.500000,0.125000,0.143229,0.156250,...,0.156250,5,Latin,Latin,loan,1,3.594073,0.913684,0.794811,Romanian-Hungarian
47381,Hădărău,Hadaró,hədərəu,hɒdɒroː,Hădărău,Hadaró,0.571429,0.142857,0.181548,0.196429,...,0.196429,4,Latin,Latin,loan,1,0.199825,0.855007,0.712788,Romanian-Hungarian
47382,Mateiaș,Mátyás,mateiaʃ,maːcaːʃ,Mateiaș,Matthias,0.571429,0.428571,0.288690,0.321429,...,0.318452,5,Latin,Latin,loan,1,-0.000949,0.877873,0.582590,Romanian-Hungarian
